# Détection des crises sanitaires

#### Scrapping du site Yelp

    Yelp n'est pas forcément le meilleur site pour récupérer des commentaires en français, mais ce code est une base pour développer le même scraping vers d'autres sites

In [27]:
from bs4 import BeautifulSoup
import urllib.request, re, pandas
import csv

#### Le résultat attendu

In [2]:
pandas.DataFrame(columns = ["Resto","Adresse","Code Postal","Ville",'Note',"Date du commentaire",'Commentaire'])

,Resto,Adresse,Code Postal,Ville,Note,Date du commentaire,Commentaire


In [4]:
def get_requests_yelp (url, nb, niveau) : #Peut-être réutilisable pour itérer de resto en resto
    """A partir de la première page du restaurant,on regarde le nombre de pages de commentaires pour ce restaurant,
    pour pouvoir itérer sur les différents questions. On doit observer le nb de commentaires par page pour savoir
    comment formuler la requête. Dans yelp c'est url?start=k*nb pour la k-1eme page.
    Retourne une liste des url pour lesquelles on pourra se servir de scrap_page_yelp"""
    if "adredir" in url : 
        return []
    while True : #Voir ligne 32
        try :
            furl = urllib.request.urlopen(url)#ça nous fait ouvrir la première page dans chaque fonction, à améliorer
            break
        except Exception as e : 
            print(e)
            continue
    soup = BeautifulSoup(furl, 'html.parser')
    try : last_page = int(soup.find(class_ = "page-of-pages arrange_unit arrange_unit--fill").get_text().split('sur')[-1])
    except : return [] #Pour le cas où on a aucune review
    list_requests = [url]
    for i in range(1,last_page) : #On ignore la première
        if niveau == "global" : #la requête ?start =i ne se place pas au même endroit selon si on est dans 
                                #un restaurant ou dans la liste des restaurants
            list_requests.append((url.split('&')[0]+'&start=%i&'%(i*nb)+url.split('&')[-1]))
        else :  
            list_requests.append((url+("?start=%i"%(i*nb))))
    return list_requests

def scrap_page_yelp (url) : 
    '''Scrape sur une page les commentaires qu il trouve, en se servant des balises HTML
    Retourne un dataframe avec en colonnes le commentaire, la note et le resto dont c'est issu'''
    scraped_page = pandas.DataFrame(columns = ["Resto","Adresse","Code Postal",
                                                "Ville",'Note',"Date du commentaire",'Commentaire'])
    stop = 0
    while True : #Permet de relancer une demande si on atteint le timeout
        try :
            furl = urllib.request.urlopen(url, timeout = 5)
            soup = BeautifulSoup(furl, 'html.parser')
            resto = (soup.h1).get_text().strip()
            try :
                Adresse = (soup.find(itemprop = "streetAddress")).get_text()
                CP = (soup.find(itemprop="postalCode")).get_text()
            except : 
                Adresse = 0
                CP = 0
                #Bâtard de camion qui fume
            Ville = (soup.find(itemprop="addressLocality")).get_text()
            for review in soup.find_all(class_ = 'review-content'):
                Date = review.find(itemprop = "datePublished")['content']
                if Date[:4] != "2016" : #On ne prend que les commentaires de 2016, les plus anciens ont peu d'intérêt
                    stop = 1
                    break 
                note = float(review.meta['content']) #Je sais pas si les demi-points existent, dans le doute, on met float
                comment = review.p.get_text() #Get_text nous permet de nous passer des balises
                scraped_page = scraped_page.append({'Resto': resto,"Adresse" : Adresse, "Code Postal" : CP,
                                                    'Note': note, "Date du commentaire" : Date, 
                                                    "Ville" : Ville,'Commentaire': comment},
                                                   ignore_index = True)#Sans ignore_index on a une syntax Error
            return (scraped_page, stop)
        except Exception as e : #On voit pourquoi ça ne marche pas
            print(e)
            continue

def scrap_resto (url) :
    if "biz" not in url : return #Cas des pages de pubs interstitielles
    requests = get_requests_yelp(url, 20, "resto")
    if len(requests) < 1 : return #Cas où on a 0 commentaires
    print("scraping...",url)
    df_resto = pandas.DataFrame(columns = ["Resto","Adresse","Code Postal",
                                        "Ville",'Note',"Date du commentaire",'Commentaire'])
    for page in requests :
        scrapped_page = scrap_page_yelp(url)
        df_resto = df_resto.append(scrapped_page[0])
        if scrapped_page[1] == 1 : break #A partir du moment où on a un commentaire antérieur à 2016, 
                                        #tous les suivants le seront aussi
    return(df_resto)

In [6]:
dataframe_general = pandas.DataFrame(columns = ["Resto","Adresse","Code Postal", "Ville",'Note',"Date du commentaire",'Commentaire'])
starting_url = "http://www.yelp.fr/search?find_loc=Paris&cflt=food"
starting_furl = urllib.request.urlopen(starting_url)
request_restos = get_requests_yelp(starting_url, 10, "global")
request_restos

['http://www.yelp.fr/search?find_loc=Paris&cflt=food',
 'http://www.yelp.fr/search?find_loc=Paris&start=10&cflt=food',
 'http://www.yelp.fr/search?find_loc=Paris&start=20&cflt=food',
 'http://www.yelp.fr/search?find_loc=Paris&start=30&cflt=food',
 'http://www.yelp.fr/search?find_loc=Paris&start=40&cflt=food',
 'http://www.yelp.fr/search?find_loc=Paris&start=50&cflt=food',
 'http://www.yelp.fr/search?find_loc=Paris&start=60&cflt=food',
 'http://www.yelp.fr/search?find_loc=Paris&start=70&cflt=food',
 'http://www.yelp.fr/search?find_loc=Paris&start=80&cflt=food',
 'http://www.yelp.fr/search?find_loc=Paris&start=90&cflt=food',
 'http://www.yelp.fr/search?find_loc=Paris&start=100&cflt=food',
 'http://www.yelp.fr/search?find_loc=Paris&start=110&cflt=food',
 'http://www.yelp.fr/search?find_loc=Paris&start=120&cflt=food',
 'http://www.yelp.fr/search?find_loc=Paris&start=130&cflt=food',
 'http://www.yelp.fr/search?find_loc=Paris&start=140&cflt=food',
 'http://www.yelp.fr/search?find_loc=Paris&s

In [25]:
dataframe_general1 = pandas.DataFrame(columns = ["Resto","Adresse","Code Postal", "Ville",'Note',"Date du commentaire",'Commentaire'])
for request_resto in request_restos[33:43] :
    print(request_resto)
    restosoup = BeautifulSoup(urllib.request.urlopen(request_resto, timeout = 5),'html.parser')
    for j in restosoup.find_all(class_ = "biz-name") :
        url_resto = 'http://www.yelp.fr'+j['href']
        dataframe_general1 = dataframe_general1.append(scrap_resto(url_resto))

http://www.yelp.fr/search?find_loc=Paris&start=330&cflt=food


HTTPError: HTTP Error 503: Service Temporarily Unavailable

In [10]:
dataframe_general.head()

,Resto,Adresse,Code Postal,Ville,Note,Date du commentaire,Commentaire
0,Café Charlot,38 rue de Bretagne,75003,Paris,4,2016-02-17,Endroit branché et sympa !!! Allez y !!! Le th...
1,Café Charlot,38 rue de Bretagne,75003,Paris,3,2016-01-30,C'est tout a fait au hasard que je me suis ret...
0,Boot Café,19 rue du Pont aux Choux,75003,Paris,5,2016-03-03,Un vrai petit bonheur ce coffee shop !C'est mi...
1,Boot Café,19 rue du Pont aux Choux,75003,Paris,4,2016-03-11,On dit souvent que tout ce qui est petit est m...
0,Ladurée,21 rue Bonaparte,75006,Paris,5,2016-01-26,Un de mes lieux favoris sur Paris. On est accu...


In [32]:
dataframe_general.to_csv("restos_330to__.csv", sep=",")